In [7]:
import sys
import os
import time
import json


import numpy as np
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.manifold import TSNE

from sklearn import tree

In [8]:
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/test.csv'
test_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")
url = 'https://raw.githubusercontent.com/malogalbo/BasicTitanic/main/data/train.csv'
train_data = pd.read_csv(url, encoding='cp1252', index_col = 0, on_bad_lines = "skip")

In [9]:
test_data = test_data.to_numpy()
train_data = train_data.to_numpy()



# Loading and Pre-Processing Data

In [10]:
#Main Cell

[[0 3 'Braund, Mr. Owen Harris' ... 7.25 nan 'S']
 [1 1 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)' ... 71.2833
  'C85' 'C']
 [1 3 'Heikkinen, Miss. Laina' ... 7.925 nan 'S']
 ...
 [0 3 'Johnston, Miss. Catherine Helen "Carrie"' ... 23.45 nan 'S']
 [1 1 'Behr, Mr. Karl Howell' ... 30.0 'C148' 'C']
 [0 3 'Dooley, Mr. Patrick' ... 7.75 nan 'Q']]


In [11]:
def train_val_split(x, y, prop_vec, shuffle=True, seed=None):

    assert x.shape[0] == y.shape[0]
    prop_vec = prop_vec / np.sum(prop_vec) # normalize

    n = x.shape[0]
    n_train = int(np.ceil(n * prop_vec[0]))
    n_val = n - n_train

    assert np.amin([n_train, n_val]) >= 1   

    if shuffle:
        rng = np.random.default_rng(seed)
        pi = rng.permutation(n)
    else:
        pi = np.arange(0, n)

    pi_train = pi[0:n_train]
    pi_val = pi[n_train:n]

    train_x = x[pi_train]
    train_y = y[pi_train]

    val_x = x[pi_val]
    val_y = y[pi_val]  
    
    return train_x, train_y, val_x, val_y

One Hot Encode:
"Pclass" 3rd, "Embarked" 

From String to Bool:
"Sex" 5th

Normalize/Standardize
"Fare"
"Age"
"Parch"
"sibsp"


Ignore/Rid 
"Name" 4th
PassengerID 1st
"ticket"


Perhaps?
"Cabin"


y-column:
"Survived" 2nd

In [12]:
# Clean According To Above

# Extract y-column from training

# Then Split